# *N*-grammes et collocations


## Définitions

### *N*-grammes

En traitement automatique du langage naturel, un *n*-gramme représente simplement une séquence de plusieurs mots.

Pour ne s'attacher qu’aux séquences de deux mots, on dénombre cinq bigrammes dans l’énoncé suivant :

```txt
(1) Le petit chat boit du lait.
```

Ces bigrammes sont :
1. Le, petit
2. petit, chat
3. chat, boit
4. boit, du
5. du, lait

Dans cette liste, tous les bigrammes n’ont pas le même poids. Quand *boit, du* ne veut pas dire grand chose, le bigramme *petit, chat* est bien plus significatif. C’est ici qu’entrent en jeu les collocations.

### Collocations

Exprimée simplement, la collocation est le processus d’identification de deux ou plusieurs mots qui apparaissent fréquemment ensemble dans un énoncé.

Elle permet de mettre en évidence, dans un énoncé, différents phénomènes linguistiques comme :
- la lexicalisation (*au fur et à mesure*, *c’est-à-dire*)
- les tics de langage (*pas de souci*, *ou pas*, *voilà voilà*)
- les cooccurrences privilégiées (*courir vite*, *procès d’intention*, *soleil de plomb*)

Pour être élue comme collocation, la cooccurrence doit être plus fréquente dans l’énoncé que chacun des éléments qui la composent.

Par exemple, la cooccurrence *nuit noire* n’est qualifiée de collocation que si elle est plus fréquente que les termes *nuit* et *noire*.

## Lister des *n*-grammes

La librairie NLTK met à disposition des méthodes pour lister facilement les *n*-grammes dans un énoncé. Ces méthodes ayant besoin en entrée d’une liste de mots, une étape préalable de tokenisation est indispensable :

In [ ]:
from nltk.tokenize import word_tokenize

sent = "Le petit chat boit du lait."
words = word_tokenize(sent)

Pour lister les bigrammes, appeler la méthode `bigrams()` :

In [ ]:
from nltk import bigrams

bigrams = bigrams(words)
print(list(bigrams))

Même principe pour les trigrammes :

In [ ]:
from nltk import trigrams

trigrams = trigrams(words)
print(list(trigrams))

Et au-delà ? Une méthode `ngrams()` avec un paramètre `n` pour définir le *n*-gramme souhaité. Par exemple pour des tétragrammes (4-grammes) :

In [ ]:
from nltk import ngrams

tetragrams = ngrams(words, 4)
print(list(tetragrams))

Afin d’améliorer les résultats, supprimer les mots vides et la ponctuation :

In [ ]:
from nltk import bigrams
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# Regexp: selects only words
tokenizer = RegexpTokenizer(r'\w+')
# List stopwords
stopwords = stopwords.words('french')
# Tokenization
words = [
    word.lower()
    for word in tokenizer.tokenize(sent)
]

On ne sélectionne un bigramme que si aucun de ses éléments ne fait partie des mots vides :

In [ ]:
bigrams = bigrams(words)
clean_bigrams = [
    (n1, n2)
    for n1, n2 in bigrams
    if n1 not in stopwords
    and n2 not in stopwords
]

print(list(clean_bigrams))

## Détecter des collocations

Les classes `BigramCollocationFinder` et `TrigramCollocationFinder`, permettent de dénicher dans un texte les 2-grammes et 3-grammes qui forment des collocations :

In [ ]:
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

En entrée, ces classes ont besoin qu’on leur fournisse une liste de mots :

In [ ]:
from nltk.corpus import PlaintextCorpusReader

# Loading the corpus
corpus = PlaintextCorpusReader('./data', '.*', encoding='utf8')
# List of words
words = [word.lower() for word in corpus.words('salammbo.txt')]
# 2-grams collocation finder
collocations = BigramCollocationFinder.from_words(words)

### Score des *n*-grammes

Le package `nltk.metrics` offre des outils de mesure adaptés pour attribuer un score aux *n*-grammes :

In [ ]:
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures

Parmi les outils de mesure à disposition, [la fonction de vraisemblance](https://fr.wikipedia.org/wiki/Rapport_de_vraisemblance) (*likelihood ratio*) est souvent la première à utiliser :

In [ ]:
likelihood = BigramAssocMeasures.likelihood_ratio

Et pour obtenir ensuite les *n* bigrammes les plus fréquents, appeler la méthode `nbest()` :

In [ ]:
collocations.nbest(likelihood, 5)

La méthode `score_ngrams()` permet de connaître le score attribué aux *n*-grammes :

In [ ]:
collocations.score_ngrams(likelihood)[:5]

Ce qui permet de limiter les résultats aux *n*-grammes qui dépassent un certain score :

In [ ]:
print(list(collocations.above_score(likelihood, 2000)))

En plus de la fonction de vraisemblance, citons quelques autres outils de mesure :
- la fréquence d’apparition
- la [PMI](https://en.wikipedia.org/wiki/Pointwise_mutual_information) (*Pointwise mutual information*)
- le [test de Student](https://fr.wikipedia.org/wiki/Test_de_Student)
- le [test du $χ^2$](https://fr.wikipedia.org/wiki/Test_du_%CF%87%C2%B2)

In [ ]:
print(f"Fréquence d’apparition : {collocations.nbest(BigramAssocMeasures.raw_freq, 5)}")
print(f"PMI : {collocations.nbest(BigramAssocMeasures.pmi, 5)}")
print(f"Test t : {collocations.nbest(BigramAssocMeasures.student_t, 5)}")
print(f"Khi carré : {collocations.nbest(BigramAssocMeasures.chi_sq, 5)}")

### Filtrer les résultats

Les résultats ne sont pas très probants : ponctuations et mots vides ressortent comme les plus fréquents. Une méthode `apply_word_filter()` ajoute un filtre sur les mots sélectionnés :

In [ ]:
from nltk.corpus import stopwords

# List of stopwords
stopwords = stopwords.words('french')
# A filter calls a lambda function
filter_stopwords = lambda w: w in stopwords
collocations.apply_word_filter(filter_stopwords)

Même si la situation s’améliore, les signes de ponctuation perturbent encore les résultats :

In [ ]:
collocations.nbest(likelihood, 5)

La solution consiste à modifier le filtre afin de supprimer les mots de un ou deux caractères :

In [ ]:
filter_stopwords = lambda w: w in stopwords or len(w) < 3
collocations.apply_word_filter(filter_stopwords)
collocations.nbest(likelihood, 5)

Il est également possible d’imposer une fréquence d’apparition minimale à un *n*-gramme grâce à la méthode `apply_freq_filter()` :

In [ ]:
collocations.apply_freq_filter(3)
collocations.nbest(likelihood, 5)